In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from faiss_module import make_db
import pandas as pd
def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    # context = "<|start_header_id|>system<|end_header_id|>\nContext\n"
    for i, doc in enumerate(docs):
        #context += f"Document {i+1}\n"
        context += doc.page_content
        context += '\n\n'
    # context += "<|eot_id|>"
    return context 

def format_docs_wnum(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    # context = "<|start_header_id|>system<|end_header_id|>\nContext\n"
    for i, doc in enumerate(docs):
        context += f"Document {i+1}\n"
        context += doc.page_content
        context += '\n\n'
    # context += "<|eot_id|>"
    return context

c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 파인튜닝 데이터셋 만들기

### Train Set

In [2]:
train_df = pd.read_csv('train.csv')
train_db = make_db(train_df,'./train_faiss_db')
answer_list = []
context_list = []
context_list_wnum = []
for i, entry in enumerate(train_df.to_dict(orient='records')):
    question = entry['Question']
    answer = entry['Answer']+"<|eot_id|>"
    # print(question)
    # print(answer)
    train_retriever = train_db.as_retriever(search_type="similarity_score_threshold",
                search_kwargs={'filter': {'source':entry['Source_path']},'score_threshold': 0.76,'k':3})
    docs = train_retriever.invoke(question)
    if len(docs) == 0:
        context_list.append("None")
        context_list_wnum.append("None")
    else:
        #print(format_docs(docs))
        context_list.append(format_docs(docs))
        context_list_wnum.append(format_docs_wnum(docs))
    answer_list.append(answer)
    
train_df['Answer'] = answer_list
train_df['context'] = context_list
train_df['context_wnum'] = context_list_wnum


Loading FAISS DB from: ./train_faiss_db


c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.76
  warnings.warn(
c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.76
  warnings.warn(
c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.76
  warnings.warn(
c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\langchain

In [3]:
# 프롬프트 생성
def create_prompt(row):
    context = row['context']
    question = row['Question']
    prompt  =f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|> 
You are a Korean Q&A Assistant.<|eot_id|>
<|start_header_id|>system<|end_header_id|>
{context}
<|start_header_id|>user<|end_header_id|>
{question}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""
    return prompt

train_df['prompt'] = train_df.apply(create_prompt, axis=1)

In [4]:
train_df.head()

,SAMPLE_ID,Source,Source_path,Question,Answer,context,context_wnum,prompt
0,TRAIN_000,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?,"2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년...",주요 재정통계●\nⅠ.\n201재정체계\n▸중앙정부 재정체계는 예산(일반 ･특별회계...,Document 1\n주요 재정통계●\nⅠ.\n201재정체계\n▸중앙정부 재정체계는...,<|begin_of_text|><|start_header_id|>system<|en...
1,TRAIN_001,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요?,"2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7...",주요 재정통계●\nⅠ.\n201재정체계\n▸중앙정부 재정체계는 예산(일반 ･특별회계...,Document 1\n주요 재정통계●\nⅠ.\n201재정체계\n▸중앙정부 재정체계는...,<|begin_of_text|><|start_header_id|>system<|en...
2,TRAIN_002,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,기금이 예산과 다른 점은?,"기금은 예산과 구분되는 재정수단으로서 재정운영의 신축성을 기할 필요가 있을 때, 정...",∙융자사업 등 기금 고유사업 수행\n확정 절차∙부처의 예산 요구\n∙기획재정부의 정...,Document 1\n∙융자사업 등 기금 고유사업 수행\n확정 절차∙부처의 예산 요...,<|begin_of_text|><|start_header_id|>system<|en...
3,TRAIN_003,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,"일반회계, 특별회계, 기금 간의 차이점은 무엇인가요?","일반회계는 특정 사업 운영 및 특정 세입으로 특정 세출을 충당하는데 사용되고, 특별...",특별회계49.6 47.2 51.3 53.6 59.1 59.7 71.6 81.7\n(...,Document 1\n특별회계49.6 47.2 51.3 53.6 59.1 59.7 ...,<|begin_of_text|><|start_header_id|>system<|en...
4,TRAIN_004,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,"2024년 총수입은 얼마이며, 예산수입과 기금수입은 각각 몇 조원인가요?","2024년 총수입은 612.2조원이며, 예산수입은 395.5조원, 기금수입은 216...","주요 재정통계●\nⅠ.\n402재정수입\n▸2024년도 총수입은 612.2조원이며,...",Document 1\n주요 재정통계●\nⅠ.\n402재정수입\n▸2024년도 총수입...,<|begin_of_text|><|start_header_id|>system<|en...


In [5]:
from sklearn.model_selection import train_test_split

# 데이터셋을 학습셋과 검증셋으로 나누기
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=52)

# train_df는 학습셋, val_df는 검증셋

### 허깅페이스 데이터셋 만들기

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
from peft import get_peft_model, LoraConfig, TaskType

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_use_double_quant=True, 
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=torch.bfloat16
)
#만약 GPU에서 amp(Automatic Mixed Precision)를 지원한다면, fp16 대신 bfloat16을 사용하는 것이 더욱 안정적일 수 있습니다.

# 모델 로드 및 양자화 설정 적용
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# LoRA 어댑터 설정 추가
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    inference_mode=False, 
    r=8, 
    lora_alpha=32, 
    lora_dropout=0.1
)
# task_type: 적용할 작업의 유형(GPT 계열의 경우 CAUSAL_LM).
# inference_mode: 학습 모드(False) 또는 추론 모드(True).
# r: LoRA의 병목 차원.
# lora_alpha: 스케일링 인자.
# lora_dropout: 드롭아웃 확률.

# 모델에 LoRA 어댑터 적용
model = get_peft_model(model, peft_config)

# 이후 Trainer를 사용해 파인튜닝을 진행


`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 4/4 [00:12<00:00,  3.05s/it]


In [7]:
from datasets import Dataset
# 토크나이즈 함수 정의
def tokenize_function(examples):
    return tokenizer(examples['prompt'], padding='max_length', truncation=True, max_length=512)
# 모델의 정답 라벨 설정
def add_labels(examples):
    labels = tokenizer(examples['Answer'], padding='max_length', truncation=True, max_length=512).input_ids
    examples['labels'] = labels
    return examples

# 데이터셋 생성
train_dataset = Dataset.from_pandas(train_df[['prompt', 'Answer']])
# 데이터셋에 토크나이즈 적용
tokenized_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.map(add_labels, batched=True)
# 검증셋에도 동일하게 적용
tokenized_eval_dataset = Dataset.from_pandas(val_df[['prompt', 'Answer']])
tokenized_eval_dataset = tokenized_eval_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = tokenized_eval_dataset.map(add_labels, batched=True)

Map: 100%|██████████| 100/100 [00:00<00:00, 9070.92 examples/s]


### 모델 학습

In [8]:
from collections import Counter
import numpy as np
# F1 점수를 계산하는 함수
def calculate_f1_score(true_sentence, predicted_sentence, sum_mode=True):
    true_sentence = ''.join(true_sentence.split())
    predicted_sentence = ''.join(predicted_sentence.split())
    
    true_counter = Counter(true_sentence)
    predicted_counter = Counter(predicted_sentence)
    
    if sum_mode:
        true_positive = sum((true_counter & predicted_counter).values())
        predicted_positive = sum(predicted_counter.values())
        actual_positive = sum(true_counter.values())
    else:
        true_positive = len((true_counter & predicted_counter).values())
        predicted_positive = len(predicted_counter.values())
        actual_positive = len(true_counter.values())

    precision = true_positive / predicted_positive if predicted_positive > 0 else 0
    recall = true_positive / actual_positive if actual_positive > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1_score
# compute_metrics 함수 정의
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    predicted_sentences = [tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions]
    true_sentences = [tokenizer.decode(label, skip_special_tokens=True) for label in labels]
    
    f1_scores = [calculate_f1_score(true, pred)[2] for true, pred in zip(true_sentences, predicted_sentences)]
    avg_f1_score = np.mean(f1_scores)
    
    return {"f1": avg_f1_score}

In [9]:
from transformers import Trainer, TrainingArguments
torch.cuda.empty_cache()
# 파인튜닝 설정
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",  # 에포크마다 검증을 수행하도록 설정
    gradient_accumulation_steps=1,
    learning_rate=5e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16 = True  # Mixed Precision을 사용하여 메모리 최적화
)

# Trainer 선언
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 파인튜닝 실행
trainer.train()

 33%|███▎      | 396/1188 [03:36<07:11,  1.83it/s]We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


OutOfMemoryError: CUDA out of memory. Tried to allocate 9.05 GiB. GPU 

In [ ]:
# 모델 저장
model.save_pretrained("./saved_model")q
tokenizer.save_pretrained("./saved_model")

from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# 양자화된 모델과 LoRA 어댑터가 적용된 모델 로드
# model = AutoModelForCausalLM.from_pretrained("./saved_model", quantization_config=bnb_config)
# model = AutoModelForCausalLM.from_pretrained("./saved_model", quantization_config=bnb_config)
# model = PeftModel.from_pretrained(model, "./saved_model")

### Test Set